@haochenz96 05/29/2023
This demo notebook will show how to filter the raw {sample}.mpileup.h5 using the matched CRAVAT dataframe.

The {sample}.mpileup.h5 is output by the TapVarCallSmk pipeline; the matched CRAVAT dataframe contains annotation provided by https://opencravat.org/, as well as matched bulk sequencing data (normal and tumor)

In [35]:
from pathlib import Path
import mosaic.io as mio
import pandas as pd
import numpy as np

# SNV filtering
from tea.cravat import get_technical_artifact_mask

# SNV plotting
from tea.format import isNaN
from tea.plots import plot_snv_clone

In [29]:
# load example CRAVAT dataframe and H5

sample_i = 'RA17_22-39_6'

cravat_f = '../data/RA17_22/RA17_22-39_6_CRAVAT_output.cleaned.txt'
cravat_df = pd.read_csv(
    cravat_f, sep='\t', index_col=0, header=[0,1]
)

sample_obj = mio.load("../data/RA17_22/RA17_22-39_6.mpileup.h5")
sample_obj.dna.genotype_variants(
    min_dp = 8, min_alt_read = 3, assign_low_conf_genotype = True,
)

Loading, ../data/RA17_22/RA17_22-39_6.mpileup.h5


Loaded in 1.0s.


# filter out technical artifacts

first we filter out technical artifacts based on:

1. the SNV is labeled as base-quality (Mutect2) in a certain number cells
2. the SNV recurrently appears in normal samples (either Broad Institute bulk whole-exome panel of normals, or this Tapestri panel-specific panel of normals). A caveat of this is that population-frequent SNPs might be filtered out. Therefore we rescue SNVs present above a certain frequency in 1000 genomes.

In [12]:
num_cells = sample_obj.dna.shape[0]

# all params are set to default
technical_mask = get_technical_artifact_mask(
    cravat_df, 
    num_cells = num_cells, 
    bq_prev_threshold = 0.005, 
    normals_pon_occurence = 4, 
    rescue_1000genome_af = 0.01,  
    filter_broad_wes_pon = False
    )
    

# get SNVs that are validated by bulk

Then we select SNVs that are detected by bulk sequencing in the matched normal/tumor sample. 

If an SNV is detected by bulk in matched normal, it is considered a germline SNP;
if an SNV is detected by bulk in matched tumor, it is considered a somatic tumor-related SNV.

In [21]:
bulk_mask = (cravat_df[('bulk_comparison', 'bulk-matched_bulk_normal-AF')] > 0) | (cravat_df[('bulk_comparison', 'bulk-matched_bulk_cohort-AF')] > 0)


bulk_germline_snv_set = set(cravat_df[(cravat_df[('bulk_comparison', 'bulk-matched_bulk_normal-AF')] > 0) & technical_mask].index)

bulk_somatic_snv_set = set(cravat_df[(cravat_df[('bulk_comparison', 'bulk-matched_bulk_cohort-AF')] > 0) & technical_mask].index)

# Plot final list of SNVs

In [27]:
voi = cravat_df[technical_mask & bulk_mask].index.tolist()
len(voi)

70

## Create an annotation map, with colors differentiating between germline vs somatic SNVs

In [28]:
ann = cravat_df.loc[voi, :].index.map(
    lambda x: 
    cravat_df.loc[x, ('Variant Annotation', 'Gene')] + ' ' + cravat_df.loc[x, ('Variant Annotation', 'Protein Change')] if not isNaN(cravat_df.loc[x, ('Variant Annotation','Protein Change')])
    else cravat_df.loc[x, ('Variant Annotation','Gene')] + ' ' + cravat_df.loc[x, ('Variant Annotation','Sequence Ontology')]
)

ann_map = dict(zip(voi, ann))

germline_var_col = '#00cc66' # dark green
somatic_var_col = '#ff0000' # red

for var_i in ann_map:
    # germline
    if var_i in bulk_germline_snv_set:
        ann_map[var_i] = f'<span style="color:{germline_var_col};">' + ann_map[var_i] + '</span>'
    elif var_i in bulk_somatic_snv_set:
        ann_map[var_i] = f'<span style="color:{somatic_var_col};">' + ann_map[var_i] + '</span>'
    else:
        pass

In [30]:
fig = plot_snv_clone(
        sample_obj,
        sample_name=sample_i,
        story_topic = 'bulk_snvs',
        voi = voi,
        attribute = 'AF_MISSING',
        ann_map = ann_map
    )

In [36]:
fig.update_layout(
    width = len(voi) * 20,
)

Path('figures').mkdir(exist_ok=True)
fig.write_image(
    f'figures/{sample_i}_bulk_snvs.pdf',
)